In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:

import json, shutil
from uuid import uuid4
from datetime import datetime, timedelta
from collections import Counter

# ADK imports 
from google.adk.agents import LlmAgent
from google.adk.runners import InMemoryRunner
from kaggle_secrets import UserSecretsClient

# Load API key (set in Kaggle Secrets)
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

print("Imports done, API key set.")


Imports done, API key set.


In [3]:
# Minimal recipes CSV 
recipes = [
    {"name": "Veggie Pasta", "diet": "vegetarian", "recipe_text": "pasta, tomato, basil, olive oil. Boil pasta, make sauce, mix."},
    {"name": "Chicken Stir Fry", "diet": "none", "recipe_text": "chicken, mixed veggies, soy sauce. Stir fry chicken, add veggies."},
    {"name": "Avocado Toast", "diet": "vegetarian", "recipe_text": "bread, avocado, salt. Toast bread, mash avocado, spread."}
]
recipes_df = pd.DataFrame(recipes)
recipes_path = "/kaggle/working/recipes.csv"
recipes_df.to_csv(recipes_path, index=False)
print("recipes.csv created at", recipes_path)

# Shared state for agents
STATE_PATH = "/kaggle/working/sharey_multi_state.json"
initial_state = {
    "members": ["John", "Maya", "Sam"],
    "chores": ["kitchen_clean","trash","bathroom_clean"],
    "assignments": [],
    "grocery_list": [],
    "meal_plan": []
}

def save_state(state):
    with open(STATE_PATH, "w") as f:
        json.dump(state, f, indent=2)

def load_state():
    if os.path.exists(STATE_PATH):
        with open(STATE_PATH, "r") as f:
            return json.load(f)
    save_state(initial_state)
    return initial_state

# initialize
save_state(initial_state)
print("Initial state saved.")

recipes.csv created at /kaggle/working/recipes.csv
Initial state saved.


In [4]:

state = load_state()

def add_assignment(chore_id, member):
    state = load_state()
    a = {"id": uuid4().hex, "chore": chore_id, "member": member,
         "assigned_at": datetime.utcnow().isoformat(), "completed_at": None}
    state["assignments"].append(a)
    save_state(state)
    return a

def complete_assignment(assignment_id):
    state = load_state()
    for a in state["assignments"]:
        if a["id"] == assignment_id:
            a["completed_at"] = datetime.utcnow().isoformat()
            save_state(state)
            return a
    return None

def add_grocery_items(items):
    state = load_state()
    for it in items:
        if it not in state["grocery_list"]:
            state["grocery_list"].append(it)
    save_state(state)
    return state["grocery_list"]

def store_meal_plan(plan):
    state = load_state()
    state["meal_plan"] = plan
    save_state(state)
    return plan


In [5]:

# Meal Planner Agent
meal_agent = LlmAgent(
    name="sharey_meal_planner",
    instruction="""
You are Sharey-Meal, a meal planning assistant for shared housing.
Input: user request + dietary preferences + available recipes context.
Output: structured JSON with 'weekly_plan' (list of days with breakfast/lunch/dinner)
and 'grocery_list' (deduplicated).
Do NOT mention being an LLM. Be concise.
""",
    model="gemini-2.5-flash"
)
meal_runner = InMemoryRunner(agent=meal_agent)

# Chore Manager Agent
chore_agent = LlmAgent(
    name="sharey_chore_manager",
    instruction="""
You are Sharey-Chore, responsible for assigning chores fairly.
Input: request to assign chores and state of past assignments.
Output: JSON with action 'assignments' and payload: list of {chore, member}.
Prefer round-robin fairness. Do not mention being an LLM.
""",
    model="gemini-2.5-flash"
)
chore_runner = InMemoryRunner(agent=chore_agent)

# Grocery Manager Agent
grocery_agent = LlmAgent(
    name="sharey_grocery_manager",
    instruction="""
You are Sharey-Grocery. Given dishes in a meal plan, return a consolidated grocery list.
Input: meal plan or list of recipes.
Output: JSON: { "grocery_list": ["item1", "item2", ...] }
Do not be verbose, do not mention being an LLM.
""",
    model="gemini-2.5-flash"
)
grocery_runner = InMemoryRunner(agent=grocery_agent)

print("All agents and runners initialized.")


All agents and runners initialized.


In [6]:

recipes_df = pd.read_csv(recipes_path)
rag_context = "\n\n".join(recipes_df["name"] + " (" + recipes_df["diet"] + "): " + recipes_df["recipe_text"])
print("RAG context ready. Example (truncated):")
print(rag_context[:400])


RAG context ready. Example (truncated):
Veggie Pasta (vegetarian): pasta, tomato, basil, olive oil. Boil pasta, make sauce, mix.

Chicken Stir Fry (none): chicken, mixed veggies, soy sauce. Stir fry chicken, add veggies.

Avocado Toast (vegetarian): bread, avocado, salt. Toast bread, mash avocado, spread.


In [7]:

import re

async def call_agent_and_parse(runner, prompt_text):
    resp = await runner.run_debug(prompt_text)
    text = resp if isinstance(resp, str) else str(resp)
    # try to extract the first JSON object
    try:
        m = re.search(r"(\{.*\}|\[.*\])", text, flags=re.S)
        if m:
            return json.loads(m.group(1)), text
    except Exception:
        pass
    return {"text": text}, text

async def orchestrate_meal_and_grocery(user_request, dietary_prefs):
    # Build prompt for meal agent including RAG context and dietary preferences
    prompt = f"User request: {user_request}\nDIETARY: {json.dumps(dietary_prefs)}\nRECIPES:\n{rag_context}\nProduce JSON with keys 'weekly_plan' and 'grocery_items'."
    meal_json, meal_text = await call_agent_and_parse(meal_runner, prompt)
    # If meal_json lacks grocery_items, call grocery agent
    grocery_items = meal_json.get("grocery_items") or []
    if not grocery_items:
        # ask grocery agent to consolidate from weekly_plan or recipes
        gp = json.dumps(meal_json.get("weekly_plan", {}))
        gprompt = f"Given these planned dishes: {gp}\nReturn JSON {{'grocery_list': [...]}}"
        gjson, gtext = await call_agent_and_parse(grocery_runner, gprompt)
        grocery_items = gjson.get("grocery_list", [])
    # store meal plan and grocery list
    store_meal_plan(meal_json.get("weekly_plan", []))
    add_grocery_items(grocery_items)
    return {"meal_plan": meal_json.get("weekly_plan", []), "grocery_list": grocery_items, "llm_texts": {"meal": meal_text}}

async def orchestrate_chore_assignment(chore_list=None):
    if chore_list is None:
        state = load_state()
        chore_list = state["chores"]
    # send simple prompt to chore agent
    prompt = f"Assign these chores fairly: {chore_list}\nCurrent members: {json.dumps(load_state()['members'])}\nReturn JSON: {{ 'assignments': [ {{'chore':'', 'member':''}}, ... ] }}."
    cjson, ctext = await call_agent_and_parse(chore_runner, prompt)
    assignments = cjson.get("assignments", [])
    applied = []
    for a in assignments:
        chore = a.get("chore")
        member = a.get("member")
        if chore and member:
            applied.append(add_assignment(chore, member))
    return {"applied": applied, "llm_text": ctext}


In [8]:
# run meal + grocery orchestrator
user_request = "Plan meals for next 3 days for 3 roommates: John needs high-protein, Maya vegetarian, Sam no restrictions."
dietary_prefs = {"John":"high-protein","Maya":"vegetarian","Sam":"no restrictions"}

result = await orchestrate_meal_and_grocery(user_request, dietary_prefs)

print("MEAL PLAN :", result["meal_plan"])
print("GROCERY LIST", result["grocery_list"])

# run chore assignment orchestrator
chore_result = await orchestrate_chore_assignment()
print("Chore assignments applied:", chore_result["applied"])



 ### Created new session: debug_session_id

User > User request: Plan meals for next 3 days for 3 roommates: John needs high-protein, Maya vegetarian, Sam no restrictions.
DIETARY: {"John": "high-protein", "Maya": "vegetarian", "Sam": "no restrictions"}
RECIPES:
Veggie Pasta (vegetarian): pasta, tomato, basil, olive oil. Boil pasta, make sauce, mix.

Chicken Stir Fry (none): chicken, mixed veggies, soy sauce. Stir fry chicken, add veggies.

Avocado Toast (vegetarian): bread, avocado, salt. Toast bread, mash avocado, spread.
Produce JSON with keys 'weekly_plan' and 'grocery_items'.
sharey_meal_planner > ```json
{
  "weekly_plan": [
    {
      "day": "Day 1",
      "breakfast": {
        "meal_name": "Avocado Toast",
        "for_john": "Avocado Toast",
        "for_maya": "Avocado Toast",
        "for_sam": "Avocado Toast"
      },
      "lunch": {
        "meal_name": "Veggie Pasta",
        "for_john": "Veggie Pasta",
        "for_maya": "Veggie Pasta",
        "for_sam": "Veggie Pa

In [9]:
import json


final_output = {
    
    "chores": chore_result,
}

# Save submission file
path = "/kaggle/working/submission.json"
with open(path, "w") as f:
    json.dump(final_output, f, indent=2)

print("Submission file created:", path)

# Verify
!ls -lh /kaggle/working


Submission file created: /kaggle/working/submission.json
total 36K
---------- 1 root root  23K Dec  1 17:44 __notebook__.ipynb
-rw-r--r-- 1 root root  284 Dec  1 17:44 recipes.csv
-rw-r--r-- 1 root root  192 Dec  1 17:44 sharey_multi_state.json
-rw-r--r-- 1 root root 1.6K Dec  1 17:44 submission.json
